#Work as SQL
import pandas as pd
import sqlite3

In [59]:
import pandas as pd 
import sqlite3

In [ ]:
#Downloader
# https://stackoverflow.com/questions/22676/how-do-i-download-a-file-over-http-using-python
import urllib2

url = "https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19Casos.csv"
# ./data/casoscovid19.csv
file_name = url.split('/')[-1]
u = urllib2.urlopen(url)
f = open(file_name, 'wb')
meta = u.info()
file_size = int(meta.getheaders("Content-Length")[0])
print "Downloading: %s Bytes: %s" % (file_name, file_size)

file_size_dl = 0
block_sz = 8192
while True:
    buffer = u.read(block_sz)
    if not buffer:
        break

    file_size_dl += len(buffer)
    f.write(buffer)
    status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
    status = status + chr(8)*(len(status)+1)
    print status,

f.close()

In [112]:
#read the CSV
df = pd.read_csv('./data/casoscovid19.csv')
#connect to a database
conn = sqlite3.connect("casoscovid19.db") #if the db does not exist, this creates a Any_Database_Name.db file in the current directory
#store your table in the database:


cursor = conn.cursor()
#Doping EMPLOYEE table if already exists
cursor.execute("DROP TABLE IF EXISTS test_0")
#print("Table dropped... ")
#Commit your changes in the database
conn.commit()
#Closing the connection
#conn.close()
# https://www.tutorialspoint.com/python_sqlite/python_sqlite_drop_table.htm

#gg = pd.read_sql('DROP TABLE IF EXISTS test_0;', conn)
#pd.read_sql_query('DROP TABLE IF EXISTS test_0;', conn)
df.to_sql('test_0', conn)

In [89]:
#connect to a database
conn = sqlite3.connect("casoscovid19.db") 
#if the db does not exist, this creates a Any_Database_Name.db file in the current directory

In [120]:
# table argentina
# Provincia	Casos Confirmados Totales	Recuperados	Fallecidos
sql_string = '''SELECT count(*) AS Total,
    sum(case when clasificacion_resumen="Confirmado" then 1 else 0 end) AS Confirmados,
    sum(case when clasificacion LIKE "%No Activo%" then 1 else 0 end) AS Recuperados,
    sum(case when clasificacion = "Caso confirmado - Fallecido" then 1 else 0 end) AS Fallecidos
FROM test_0;
'''
gg = pd.read_sql(sql_string, conn)
gg

,Total,Confirmados,Recuperados,Fallecidos
0,773272,228173,103284,4252


In [118]:
# table provincias
# Provincia	Casos Confirmados Totales	Recuperados	Fallecidos
sql_string = '''SELECT residencia_provincia_nombre  AS Provincia,
    count(*) AS Total,
    sum(case when clasificacion_resumen="Confirmado" then 1 else 0 end) AS Confirmados,
    sum(case when clasificacion LIKE "%No Activo%" then 1 else 0 end) AS Recuperados,
    sum(case when clasificacion = "Caso confirmado - Fallecido" then 1 else 0 end) AS Fallecidos
FROM test_0
GROUP BY residencia_provincia_nombre;
'''
gg = pd.read_sql(sql_string, conn)
gg

,Provincia,Total,Confirmados,Recuperados,Fallecidos
0,Buenos Aires,413892,138322,68677,2370
1,CABA,177414,66454,24220,1408
2,Catamarca,2204,60,50,0
3,Chaco,25304,3872,2866,159
4,Chubut,3471,308,174,3
5,Corrientes,4865,200,120,2
6,Córdoba,35982,2994,649,52
7,Entre Ríos,5811,999,693,11
8,Formosa,904,81,77,0
9,Jujuy,15122,2957,317,31


In [117]:
# table departamentos buenos aires
# Provincia	Casos Confirmados Totales	Recuperados	Fallecidos
sql_string = '''SELECT residencia_departamento_nombre  AS Residencia,
    count(*) AS Total,
    sum(case when clasificacion_resumen="Confirmado" then 1 else 0 end) AS Confirmados,
    sum(case when clasificacion LIKE "%No Activo%" then 1 else 0 end) AS Recuperados,
    sum(case when clasificacion = "Caso confirmado - Fallecido" then 1 else 0 end) AS Fallecidos
FROM test_0
WHERE (residencia_provincia_nombre="Buenos Aires")
GROUP BY residencia_departamento_nombre;
'''
gg = pd.read_sql(sql_string, conn)
gg

,Residencia,Total,Confirmados,Recuperados,Fallecidos
0,25 de Mayo,199,36,23,0
1,9 de Julio,287,21,14,1
2,Adolfo Alsina,35,2,2,0
3,Adolfo Gonzales Chaves,36,1,0,0
4,Alberti,158,12,7,0
...,...,...,...,...,...
131,Tres de Febrero,13810,5110,2526,107
132,Vicente López,8028,2371,1027,55
133,Villa Gesell,160,9,6,0
134,Villarino,127,19,16,0


In [116]:
# table departamentos buenos aires
# Provincia	Casos Confirmados Totales	Recuperados	Fallecidos
sql_string = '''
SELECT fecha_apertura, count(*) FROM test_0 WHERE (clasificacion_resumen="Confirmado") group by fecha_apertura  ORDER BY "fecha_apertura" DESC LIMIT 7;
'''
gg = pd.read_sql(sql_string, conn)
gg

,fecha_apertura,count(*)
0,2020-08-06,1598
1,2020-08-05,3110
2,2020-08-04,5656
3,2020-08-03,5641
4,2020-08-02,3934
5,2020-08-01,3948
6,2020-07-31,5756
